In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
import math
from numpy import dot
from numpy.linalg import norm

In [2]:
# 유튜브 구독정보 난수생성기로 임의 생성

class RandomGenerator(object):
   
    def __init__(self, random_state=1):
        """
        -매개변수-
        random_state : 가중치 초기화를 위한 무작위 난수 생성기의 seed
        """
        self.random_state = random_state
        
    def GeneratingRandomData(self, loc=4, size=5000, scale=3, n_features=10):
        """
        난수생성기로 임의 데이터 생성
        """
    
        np.random.seed(self.random_state)
        # 특정 seed를 가지는 np.random.RandomState() 객체를 만들고, 이 객체를 통해서 난수 생성
        self.data = np.random.normal(loc, scale, size)
        for i in range(len(self.data)):
            self.data[i] = (abs(int(self.data[i])))
                
        return self.data.reshape(-1, n_features)
    
    def GeneratingFeature(self): # feature 임의 생성
        self.feature = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]
    
    def ArrayToDataframe(self, random_array, n_sample):
        return pd.DataFrame(random_array, index=range(0, n_sample), columns=self.feature)

In [3]:
# 난수생성기 객체 생성

dataGenerator = RandomGenerator(156)

In [4]:
randomData = dataGenerator.GeneratingRandomData()
randomData.shape

(500, 10)

In [5]:
# feature 이름 리스트 구성

feature_names = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]

In [6]:
dataGenerator.GeneratingFeature()

In [7]:
dfRandomData = dataGenerator.ArrayToDataframe(randomData, 500)
dfRandomData

,Game,Cook,Travel,Money,Academy,Learning,Sing,Sports,Machine,Mistery
0,5.0,1.0,1.0,2.0,8.0,3.0,5.0,5.0,1.0,0.0
1,2.0,1.0,3.0,4.0,3.0,2.0,0.0,5.0,0.0,6.0
2,1.0,0.0,2.0,1.0,3.0,9.0,3.0,5.0,11.0,2.0
3,5.0,3.0,4.0,3.0,6.0,1.0,4.0,2.0,4.0,0.0
4,10.0,6.0,3.0,1.0,6.0,3.0,4.0,7.0,2.0,8.0
...,...,...,...,...,...,...,...,...,...,...
495,3.0,8.0,7.0,6.0,3.0,2.0,7.0,7.0,9.0,1.0
496,6.0,2.0,4.0,4.0,10.0,6.0,3.0,6.0,1.0,0.0
497,0.0,6.0,4.0,0.0,7.0,2.0,5.0,3.0,1.0,6.0
498,8.0,1.0,6.0,6.0,3.0,6.0,1.0,5.0,2.0,6.0


In [57]:
from sklearn.metrics.pairwise import euclidean_distances
class Classification(object):
    
    def __init__(self, df_data):
        self.df = df_data
    
    def Extraction(self):
        # 가장 선호하는 feature 추출
        self.ls = []
        for i in range(0, self.df.shape[0]):
            self.ls.append(self.df.sort_values(by=[i], axis=1).iloc[i, -2:].index.tolist())
            
        # 각 이용자의 구독 채널 수 추출
        arr = np.array(self.df)
        self.num_ch = np.sum(arr, axis=1).reshape(-1, 1)
        
    def Ohe(self): # 각 샘플에 대해 추출한 가장 선호하는 2개의 특성에 대해 원핫인코딩
        ohe = OneHotEncoder(sparse=False)
        return ohe.fit_transform(self.ls)
    
    def makeData(self, train_feature):
        self.X_train = np.hstack((self.num_ch, train_feature))
        
    def Kmeans(self): # k_means 클러스터링
        model = KMeans(n_clusters=16)
        model.fit(self.X_train)
        return np.array(model.predict(self.X_train))

'''
    def euclidean_distance(self, pt1, pt2): # 데이터 포인트 거리 측정
      #sj: numpy를 활용해 계산속도 개선
      #sj: self인자로 초기화를 해야 ReturnResult함수에서 사용가능
      distance = 0
      for i in range(len(pt1)):
        a = pt1[i] - pt2[i]
        distance += np.square(a)
      return np.sqrt(distance)
'''    

    def cosine_similarity(self, pt1, pt2):
      #sj: cosine similarity를 사용하면 결이 비슷한 사용자를 찾을 수 있음.
      #sj: 구독 채널 Id는 관련성이 있을 수록 cosine similarity가 작음.
      return dot(pt1, pt2) / (norm(pt1) * norm(pt1))
      

# sj: euclidean_distance의 속도 문제 개선 전까지는 임시로 사용 X
#     def ReturnResult_euclidean(self, k=5): 
#         # 각 이용자마다 가장 가까운 k개의 이용자 매칭 결과 반환
#         # 현재 시간복잡도가 O(n^2)이므로 계산량이 많음 -> 추후 시간복잡도 감소 필요
        
#         #k = 5
#         k_users = np.empty((0,k))
#         for i in range(0, self.X_train.shape[0]):
#             distance = []
#             for j in range(0, self.X_train.shape[0]):
#                 if i != j:
#                     distance.append(self.euclidean_distance(self.X_train[i, :], self.X_train[j, :])) #sj: euclidean_distance에 self로 초기화
#                 if j == (self.X_train.shape[0] - 1):
#                     # i 번째 데이터 포인터와 다른 데이터들 사이의 거리를 작은순으로 정렬한 후 해당 인덱스를 이용해서 가장 가까운 k명의 이용자 선별
#                     distance_idx = np.array(distance).argsort()
#                     k_users = np.append(k_users, [distance_idx[:k]], axis=0)
                    
#         return k_users


    def ReturnResult_cosine_similarity(self, k=5): 
        
        k_users = np.empty((0,k))
        for i in range(0, self.X_train.shape[0]):
            distance = []
            for j in range(0, self.X_train.shape[0]):
                if i != j:
                    distance.append(self.cosine_similarity(self.X_train[i, :], self.X_train[j, :]))
                if j == (self.X_train.shape[0] - 1):
                    # i 번째 데이터 포인터와 다른 데이터들 사이의 거리를 작은순으로 정렬한 후 해당 인덱스를 이용해서 가장 가까운 k명의 이용자 선별
                    distance_idx = np.array(distance).argsort()
                    k_users = np.append(k_users, [distance_idx[:k]], axis=0)
                    
        return k_users

In [58]:
cl = Classification(dfRandomData)

In [59]:
cl.Extraction()
cl.ls

[['Sports', 'Academy'],
 ['Sports', 'Mistery'],
 ['Learning', 'Machine'],
 ['Game', 'Academy'],
 ['Mistery', 'Game'],
 ['Academy', 'Travel'],
 ['Travel', 'Sports'],
 ['Machine', 'Mistery'],
 ['Game', 'Academy'],
 ['Machine', 'Academy'],
 ['Game', 'Learning'],
 ['Travel', 'Money'],
 ['Game', 'Travel'],
 ['Sports', 'Learning'],
 ['Learning', 'Sports'],
 ['Money', 'Machine'],
 ['Learning', 'Sing'],
 ['Learning', 'Sing'],
 ['Mistery', 'Sports'],
 ['Learning', 'Sing'],
 ['Academy', 'Machine'],
 ['Mistery', 'Sing'],
 ['Money', 'Academy'],
 ['Mistery', 'Travel'],
 ['Learning', 'Academy'],
 ['Learning', 'Machine'],
 ['Learning', 'Sing'],
 ['Travel', 'Machine'],
 ['Mistery', 'Sports'],
 ['Sing', 'Game'],
 ['Machine', 'Academy'],
 ['Mistery', 'Sports'],
 ['Cook', 'Travel'],
 ['Travel', 'Game'],
 ['Sing', 'Sports'],
 ['Game', 'Sports'],
 ['Cook', 'Mistery'],
 ['Money', 'Machine'],
 ['Cook', 'Money'],
 ['Money', 'Sing'],
 ['Travel', 'Sing'],
 ['Machine', 'Money'],
 ['Game', 'Cook'],
 ['Travel', 'L

In [60]:
train_feature = cl.Ohe()
train_feature

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.]])

In [61]:
cl.makeData(train_feature)

In [62]:
result = cl.Kmeans()
result

array([11,  6,  4,  1,  7,  8, 11, 15,  9,  2,  2,  4,  4, 15,  8, 11,  8,
       13,  1, 11,  4,  2, 10,  1, 10, 14,  1, 14,  5, 10,  5,  2, 13,  9,
        7,  2,  5,  6,  7, 11, 14, 11, 13, 14,  9,  9,  9,  1,  9,  1, 10,
        2,  4,  8,  4,  5,  4, 10,  2, 14,  4, 14,  3,  8,  0,  2, 10,  0,
        4, 15,  8, 15,  7,  9,  8,  6,  9,  1,  8, 14,  9, 12,  2,  4,  1,
        5,  5,  0,  2, 14,  1,  7,  2, 10,  2,  5,  1, 13, 10, 10,  1,  2,
        2,  9,  7,  6,  1,  1,  4, 15, 13,  1,  8, 13,  5,  5,  4, 11,  1,
        6, 15, 10, 11, 10,  5, 11, 13, 11,  4, 10,  8,  7,  1, 14,  1, 13,
        5,  6,  4,  7,  4,  3,  8,  9,  4,  9,  7,  9, 10,  7,  8, 13,  7,
        4,  5,  2,  4,  7,  9,  9,  2,  2,  7,  2,  7, 13,  7,  9,  4,  7,
        5,  6,  7,  4,  4, 13,  1, 14,  1,  9,  9, 10, 15, 14, 13,  4, 13,
        9,  0,  9,  9,  4, 13, 10,  9,  8,  9,  9,  7,  9,  5,  9,  9,  3,
        8,  6, 15,  8,  2,  9,  7,  2,  4,  8,  1,  6,  9,  5,  7,  9,  8,
        0,  3,  1, 15,  2

In [63]:
k_users = cl.ReturnResult_cosine_similarity()

In [64]:
k_users

array([[363., 454., 289.,  61., 378.],
       [454., 363.,  61., 289., 392.],
       [363., 454., 289.,  61., 378.],
       ...,
       [364., 455., 290.,  62., 397.],
       [364., 455.,  62., 290., 379.],
       [455., 364.,  62., 290., 397.]])